In [4]:
import tensorflow as tf
import PIL
from PIL import Image
import os
import matplotlib.pyplot as plt

## 5-1. 5차 학습 전 평가


### 4차 학습 모델 

- 기존 학습 방법들중 가장 성능이 좋은 모델 사용
- VGG16보다 단순한 ResNet50을 전이학습을 진행하여 구현한 모델


### 2차 데이터셋

- kill 이미지 1261
- non-kill 이미지 932


### 5차 학습 전 평가 특징

- 2차 데이터셋에서 ROI를 추출하여 평가 데이터로 사용합니다. 
- 2차 데이터셋 모두 사용하여 현재의 모델을 평가합니다. 추가된 데이터셋에 대한 정확도가 충분히 높다면 2차 데이터셋에 대한 학습이 필요하지 않기 때문입니다. 

In [1]:
ADD_IMAGE_PATH = './dataset/kill_log_set_2/'

In [52]:
add_train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    ADD_IMAGE_PATH,
    label_mode = 'binary',
    image_size = (540, 960),
)

Found 2193 files belonging to 2 classes.


In [10]:
# 대충 본다.
def crop_roi(image, label): 
    
    cropped_image = tf.image.crop_to_bounding_box(
        image, 10, 200, 190, 560
    )
    #resized_image = tf.image.resize(cropped_image, [190, 560], method = 'lanczos3')
    #casted_image = resized_image / 255
    return cropped_image, label

In [11]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

# ROI의 과정은 Data Augmentation의 수행이 아니므로 학습 데이터, 검증 데이터 모두 적용해야합니다.
def make_crop_dataset(dataset):
    dataset = dataset.cache()
    dataset = dataset.map(crop_roi, num_parallel_calls=AUTOTUNE)
    dataset = dataset.prefetch(AUTOTUNE)
    return dataset

In [10]:
add_cropped_dataset = make_crop_dataset(add_train_dataset)

In [83]:
func_saved_model_t4 = tf.keras.models.load_model('./kill_log_models/model_t4.hdf5')

In [78]:
func_saved_model_t4.evaluate(add_cropped_dataset)

69/69 [==============================] - 22s 311ms/step - loss: 0.1219 - accuracy: 0.9676


[0.1219450831413269, 0.9676242470741272]

### 5차 학습전 평가의 결과


- 평가 데이터의 정확도는 96.7%

ROI를 추출하고, 추출된 이미지를 4차 학습 모델에 대해 평가한 결과, 기존의 98.7%에서 약 2%가 감소한 것을 확인할 수 있습니다. 2%의 감소는 2차 데이터셋에 대한 학습이 충분히 유의미할 수 있을 것으로 판단됩니다.

현재 데이터가 학습과 평가 데이터가 구분되어있지 않습니다. 랜덤하게 추출하여 2차 데이터셋을 2차 데이터셋(학습용), 2차 테스트셋으로 분류합니다.

In [2]:
import os 
import numpy as np
import shutil
SPLIT_VAL = 0.2

ADD_TRAIN_PATH = './dataset/kill_log_dataset_2'
ADD_TEST_PATH = './dataset/kill_log_testset_2'

In [60]:
if not os.path.exists(ADD_TRAIN_PATH):
    os.mkdir(ADD_TRAIN_PATH)
    
if not os.path.exists(ADD_TEST_PATH):
    os.mkdir(ADD_TEST_PATH)

In [103]:
for directory_name in os.listdir(ADD_IMAGE_PATH):
    # kill_log, non_kill_log
    directory_path = os.path.join(ADD_IMAGE_PATH, directory_name)
    
    # train_directory 생성
    train_save_path = os.path.join(ADD_TRAIN_PATH, directory_name)
    if not os.path.exists(train_save_path):
        os.mkdir(train_save_path)
    print(len(os.listdir(train_save_path)))
    # test_directory 생성
    test_save_path = os.path.join(ADD_TEST_PATH, directory_name)
    if not os.path.exists(test_save_path):
        os.mkdir(test_save_path)
    
    files = os.listdir(ADD_IMAGE_PATH + directory_name)
    
    # 전체 데이터 사이즈
    data_size = len(files)
    
    # 테스트 데이터 사이즈
    test_size = int(len(files) * 0.2)
    
    # 테스트 데이터로 선택되는 인덱스
    test_data = np.random.choice(data_size, test_size, replace = False)
    
    for i, file in enumerate(files):
        if i in test_data:
            shutil.copy(os.path.join(directory_path, file), test_save_path)
        else:
            shutil.copy(os.path.join(directory_path, file), train_save_path)

#shutil.rmtree(ADD_TRAIN_PATH + '/.ipynb_checkpoints')
#shutil.rmtree(ADD_TEST_PATH + '/.ipynb_checkpoints')

1009
746


FileNotFoundError: [WinError 3] 지정된 경로를 찾을 수 없습니다: './dataset/kill_log_dataset_2/.ipynb_checkpoints'

In [5]:
add_train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    ADD_TRAIN_PATH,
    label_mode = 'binary',
    image_size = (540, 960),
)

Found 1755 files belonging to 2 classes.


In [6]:
add_test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    ADD_TEST_PATH,
    label_mode = 'binary',
    image_size = (540, 960),
)

Found 438 files belonging to 2 classes.


## 5-2. 5차 학습 


### ResNet

- 4차 학습 모델 사용
- VGG16보다 단순한 ResNet50사용


### 1차 데이터셋 (기존 학습)

- kill 이미지 786
- non-kill 이미지 1832


### 1차 테스트셋

- kill 이미지 161
- non-kill 이미지 149


### 2차 데이터셋

- kill 이미지 1009
- non-kill 이미지 746


### 2차 테스트셋

- kill 이미지 252
- non-kill 이미지 186


### 5차 학습 특징

- ROI를 수행합니다.
- 2차 테스트 셋으로 4차 학습 모델을 평가합니다.
- 그 후, 4차 학습 모델을 2차 데이터 셋으로 추가학습을 진행합니다.

In [7]:
add_train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    ADD_TRAIN_PATH,
    label_mode = 'binary',
    image_size = (540, 960),
    validation_split = 0.1,
    seed = 6309,
    subset = 'training'
)

add_val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    ADD_TRAIN_PATH,
    label_mode = 'binary',
    image_size = (540, 960),
    validation_split = 0.1,
    seed = 6309,
    subset = 'validation'
)

Found 1755 files belonging to 2 classes.
Using 1580 files for training.
Found 1755 files belonging to 2 classes.
Using 175 files for validation.


In [8]:
add_test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    ADD_TEST_PATH,
    label_mode = 'binary',
    image_size = (540, 960),
)

Found 438 files belonging to 2 classes.


In [12]:
add_cropped_trainset = make_crop_dataset(add_train_dataset)
add_cropped_valset = make_crop_dataset(add_val_dataset)
add_cropped_testset = make_crop_dataset(add_test_dataset)

In [90]:
# model에 대한 체크포인트 생성 -> 가장 좋은 모델을 저장. (모델을 저장하는 것은 ) 
checkpoint = tf.keras.callbacks.ModelCheckpoint("./kill_log_models/model_t5.hdf5", monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='auto')

In [94]:
func_model_t5 = tf.keras.models.load_model('./kill_log_models/model_t4.hdf5')
func_model_t5.fit(add_cropped_trainset, validation_data = add_cropped_valset, callbacks = [checkpoint], epochs = 10)

Epoch 1/10
44/44 [==============================] - 10s 200ms/step - loss: 0.1471 - accuracy: 0.9501 - val_loss: 0.1229 - val_accuracy: 0.9516

Epoch 00001: val_loss improved from 0.12289 to 0.12289, saving model to ./kill_log_models\model_t5.hdf5
Epoch 2/10
44/44 [==============================] - 8s 187ms/step - loss: 0.0480 - accuracy: 0.9865 - val_loss: 0.0534 - val_accuracy: 0.9801

Epoch 00002: val_loss improved from 0.12289 to 0.05337, saving model to ./kill_log_models\model_t5.hdf5
Epoch 3/10
44/44 [==============================] - 8s 187ms/step - loss: 0.0291 - accuracy: 0.9943 - val_loss: 0.0382 - val_accuracy: 0.9858

Epoch 00003: val_loss improved from 0.05337 to 0.03824, saving model to ./kill_log_models\model_t5.hdf5
Epoch 4/10
44/44 [==============================] - 8s 187ms/step - loss: 0.0190 - accuracy: 0.9964 - val_loss: 0.0369 - val_accuracy: 0.9886

Epoch 00004: val_loss improved from 0.03824 to 0.03690, saving model to ./kill_log_models\model_t5.hdf5
Epoch 5/10


In [97]:
func_model_t5.evaluate(add_cropped_testset)

14/14 [==============================] - 2s 125ms/step - loss: 0.0880 - accuracy: 0.9772


[0.08803118765354156, 0.9771689772605896]

In [99]:
func_saved_model_t5 = tf.keras.models.load_model('./kill_log_models/model_t5.hdf5')

In [100]:
func_saved_model_t5.evaluate(add_cropped_testset)

14/14 [==============================] - 2s 111ms/step - loss: 0.0880 - accuracy: 0.9772


[0.08803118765354156, 0.9771689772605896]

## 5-3. 1차 테스트 셋에 대한 평가

In [13]:
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    './dataset/kill_log_testset/',
    label_mode = 'binary',
    image_size = (540, 960),
)

Found 310 files belonging to 2 classes.


In [14]:
cropped_testset = make_crop_dataset(test_dataset)

In [101]:
func_saved_model_t5.evaluate(cropped_testset)

10/10 [==============================] - 1s 110ms/step - loss: 0.0106 - accuracy: 1.0000


[0.01064381469041109, 1.0]

### 5차 학습 결과

- 1차 평가 데이터의 정확도는 100%
- 2차 평가 데이터의 정확도는 97.7%

2차 평가 데이터에 대해서도 추가 학습 모델의 정확도가 매우 높은 것을 확인할 수 있습니다. 이전 학습에서 Data Augmentation을 통한 정확도 개선을 확인하였으므로 현재의 학습 방법에 Data Augmentation까지 추가하여 학습을 진행해보겠습니다.

## 6-1. 6차 학습


### ResNet

- 4차 학습 모델 사용
- VGG16보다 단순한 ResNet50사용


### 1차 데이터셋 (기존 학습)

- kill 이미지 786
- non-kill 이미지 1832


### 1차 테스트셋

- kill 이미지 161
- non-kill 이미지 149


### 2차 데이터셋

- kill 이미지 1009
- non-kill 이미지 746


### 2차 테스트셋

- kill 이미지 252
- non-kill 이미지 186


### 6차 학습 특징


- 4차 학습 모델을 2차 데이터 셋으로 추가학습을 진행합니다.
- ROI을 추출한 뒤 Data Augmentation을 통해 데이터셋의 크기를 늘립니다.

In [15]:
# 2번 방식의 구현
from tensorflow.keras.layers.experimental.preprocessing import RandomZoom, RandomTranslation

class AugmentationLayer(tf.keras.layers.Layer):
    def __init__(self):
        super().__init__()
        # 가로 세로          
        self.random_zoom = RandomZoom(0, (-0.4, 0), fill_mode = 'nearest')
        self.random_trans = RandomTranslation(0.2, 0, fill_mode = 'nearest')
    
    def call(self, x, y):
        x = self.random_zoom(x)
        x = self.random_trans(x)
        return x, y

In [16]:
# 1번 방식의 구현을 함께 사용합니다. 
def random_brightness(image, label):
    bright_changed_image = tf.image.random_brightness(image, 0.2, seed = 6309)
    return bright_changed_image, label

In [17]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

# map 함수를 통해 적용 -> Layer이기 때문에 모델과 함께 사용 가능.
def make_augmented_dataset(dataset):
    dataset = dataset.cache()
    dataset = dataset.map(AugmentationLayer(), num_parallel_calls=AUTOTUNE)
    dataset = dataset.map(random_brightness, num_parallel_calls=AUTOTUNE)
    dataset = dataset.prefetch(AUTOTUNE)
    return dataset

In [18]:
# Data Augmentation 적용
add_cropped_trainset = make_crop_dataset(add_train_dataset)
add_cropped_augmented_train_set = make_augmented_dataset(add_cropped_trainset)

# validation, test set은 그대로 사용
add_cropped_valset = make_crop_dataset(add_val_dataset)
add_cropped_testset = make_crop_dataset(add_test_dataset)

In [115]:
# model에 대한 체크포인트 생성 -> 가장 좋은 모델을 저장. (모델을 저장하는 것은 ) 
checkpoint = tf.keras.callbacks.ModelCheckpoint("./kill_log_models/model_t6.hdf5", monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='auto')

In [116]:
func_model_t6 = tf.keras.models.load_model('./kill_log_models/model_t4.hdf5')
func_model_t6.fit(add_cropped_augmented_train_set, validation_data = add_cropped_valset, callbacks = [checkpoint], epochs = 10)

Epoch 1/10
50/50 [==============================] - 9s 140ms/step - loss: 0.1172 - accuracy: 0.9671 - val_loss: 0.1541 - val_accuracy: 0.9314

Epoch 00001: val_loss improved from inf to 0.15412, saving model to ./kill_log_models\model_t6.hdf5
Epoch 2/10
50/50 [==============================] - 7s 131ms/step - loss: 0.0675 - accuracy: 0.9791 - val_loss: 0.0692 - val_accuracy: 0.9771

Epoch 00002: val_loss improved from 0.15412 to 0.06924, saving model to ./kill_log_models\model_t6.hdf5
Epoch 3/10
50/50 [==============================] - 7s 128ms/step - loss: 0.0480 - accuracy: 0.9880 - val_loss: 0.0561 - val_accuracy: 0.9886

Epoch 00003: val_loss improved from 0.06924 to 0.05609, saving model to ./kill_log_models\model_t6.hdf5
Epoch 4/10
50/50 [==============================] - 7s 129ms/step - loss: 0.0430 - accuracy: 0.9886 - val_loss: 0.0555 - val_accuracy: 0.9886

Epoch 00004: val_loss improved from 0.05609 to 0.05551, saving model to ./kill_log_models\model_t6.hdf5
Epoch 5/10
50/50

In [117]:
func_model_t6.evaluate(add_cropped_testset)

14/14 [==============================] - 2s 110ms/step - loss: 0.0958 - accuracy: 0.9703


[0.09581686556339264, 0.9703196287155151]

In [120]:
func_saved_model_t6 = tf.keras.models.load_model('./kill_log_models/model_t6.hdf5')
func_saved_model_t6.evaluate(add_cropped_testset)

14/14 [==============================] - 2s 110ms/step - loss: 0.0702 - accuracy: 0.9772


[0.07015789300203323, 0.9771689772605896]

In [119]:
func_saved_model_t5.evaluate(cropped_testset)

10/10 [==============================] - 1s 108ms/step - loss: 0.0106 - accuracy: 1.0000


[0.01064381469041109, 1.0]

### 6차 학습의 결과

- 학습 데이터의 가장 높은 정확도는 99%

- 1차 평가 데이터의 정확도는 100%

- 2차 평가 데이터의 정확도는 97.7%

ROI 추출 및 Data Augmentation까지 추가하여 학습을 진행해보았으나, 5차 학습과의 정확도 차이가 발생하지 않았습니다. 이로 미루어볼 때, 2차 학습 데이터를 추가학습한 5차 모델은 Data Augmentation를 통해 생성되는 새로운 이미지들에 대해서도 충분히 예측가능하다는 것으로 생각해볼 수 있습니다. 여러가지 학습 방법으로 학습을 진행해보았습니다. 가장 좋은 모델인 6차 학습 모델에 대해서 파인튜닝을 통해 정확도를 향상시키도록 하겠습니다.

## 6-2. 6차 학습에 대한 파인튜닝

평가 데이터의 정확도가 충분히 높기 때문에 파인튜닝을 통해서 현재 모델을 고도화하는 방법을 선택할 수 있습니다.


### ResNet

- 6차 학습 모델 사용
- VGG16보다 단순한 ResNet50사용


### 1차 데이터셋 

- kill 이미지 786
- non-kill 이미지 1832


### 1차 테스트셋

- kill 이미지 161
- non-kill 이미지 149


### 2차 데이터셋

- kill 이미지 1009
- non-kill 이미지 746


### 2차 테스트셋

- kill 이미지 252
- non-kill 이미지 186


### 6차 파인튜닝 특징


- 전이학습을 위해 가중치 갱신이 불가능하도록 설정한 것을 파인튜닝을 위해 가능하도록 설정합니다.
- 1, 2차의 데이터셋을 결합합니다.
- ROI을 추출한 뒤 Data Augmentation을 통해 데이터셋의 크기를 늘립니다.

In [18]:
func_model_t62 = tf.keras.models.load_model('./kill_log_models/model_t6.hdf5')

In [19]:
func_model_t62.layers[1].summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, None, None, 6 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [20]:
func_model_t62.layers[1].trainable = True

In [21]:
# Trainable params의 갯수가 증가한 것을 확인할 수 있다.
func_model_t62.layers[1].summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, None, None, 6 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [22]:
# Trainable params의 갯수가 증가한 것을 확인할 수 있다.
func_model_t62.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 190, 560, 3)]     0         
_________________________________________________________________
resnet50 (Functional)        (None, 2048)              23587712  
_________________________________________________________________
dense_10 (Dense)             (None, 32)                65568     
_________________________________________________________________
batch_normalization_5 (Batch (None, 32)                128       
_________________________________________________________________
re_lu_5 (ReLU)               (None, 32)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 33        
Total params: 23,653,441
Trainable params: 23,600,257
Non-trainable params: 53,184
__________________________________________

### 데이터셋 구성하기

- 1차 데이터셋 로드하기

In [19]:
IMAGE_PATH = './dataset/kill_log_dataset'
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    IMAGE_PATH,
    # label_mode 대신에 class_mode사용한다.
    image_size = (540, 960),
    label_mode = 'binary',
    batch_size = 30,
    seed = 6309,
    validation_split = 0.1,
    subset = 'training',
)

val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    IMAGE_PATH,
    # label_mode 대신에 class_mode사용한다.
    image_size = (540, 960),
    label_mode = 'binary',
    batch_size = 30,
    seed = 6309,
    validation_split = 0.1,
    subset = 'validation',
)

Found 2618 files belonging to 2 classes.
Using 2357 files for training.
Found 2618 files belonging to 2 classes.
Using 261 files for validation.


In [20]:
cropped_trainset = make_crop_dataset(train_dataset)
add_cropped_trainset = make_crop_dataset(add_train_dataset)

In [21]:
cropped_valset = make_crop_dataset(val_dataset)
add_cropped_valset = make_crop_dataset(add_val_dataset)

In [22]:
# 1, 2차 데이터셋이 결합된 데이터셋
merged_trainset = cropped_trainset.concatenate(add_cropped_trainset).shuffle(4000)

In [23]:
# 1, 2차 데이터셋이 결합된 데이터셋
merged_valset = cropped_valset.concatenate(add_cropped_valset).shuffle(1000)

In [24]:
# 결합된 데이터셋에 Data Augmentation 추가
merged_cropped_augmented_train_set = make_augmented_dataset(merged_trainset)

In [29]:
# val_loss에 따른 빠른 종료 -> 빠른 구현을 위해서 선택적으로 적용할 수 있다.
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

# model에 대한 체크포인트 생성 -> 가장 좋은 모델을 저장. (모델을 저장하는 것은 ) 
checkpoint = tf.keras.callbacks.ModelCheckpoint("./kill_log_models/model_t62.hdf5", monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='auto')

In [30]:
func_model_t62.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),  # Low learning rate
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=['accuracy'],
)

In [31]:
func_model_t62.fit(merged_cropped_augmented_train_set, validation_data = merged_valset, callbacks = [checkpoint], epochs = 20)

Epoch 1/20
129/129 [==============================] - 102s 478ms/step - loss: 0.0883 - accuracy: 0.9738 - val_loss: 0.0502 - val_accuracy: 0.9862

Epoch 00001: val_loss improved from inf to 0.05015, saving model to ./kill_log_models\model_t62.hdf5
Epoch 2/20
129/129 [==============================] - 52s 401ms/step - loss: 0.0460 - accuracy: 0.9831 - val_loss: 0.0401 - val_accuracy: 0.9885

Epoch 00002: val_loss improved from 0.05015 to 0.04014, saving model to ./kill_log_models\model_t62.hdf5
Epoch 3/20
129/129 [==============================] - 53s 402ms/step - loss: 0.0270 - accuracy: 0.9931 - val_loss: 0.0601 - val_accuracy: 0.9885

Epoch 00003: val_loss did not improve from 0.04014
Epoch 4/20
129/129 [==============================] - 53s 402ms/step - loss: 0.0294 - accuracy: 0.9914 - val_loss: 0.0342 - val_accuracy: 0.9885

Epoch 00004: val_loss improved from 0.04014 to 0.03422, saving model to ./kill_log_models\model_t62.hdf5
Epoch 5/20
129/129 [==============================] -

In [33]:
merged_testset = cropped_testset.concatenate(add_cropped_testset).shuffle(1000)

In [36]:
func_model_t6.evaluate(merged_testset)

NameError: name 'func_model_t6' is not defined

In [37]:
func_model_t62.evaluate(merged_testset)

24/24 [==============================] - 7s 139ms/step - loss: 0.0390 - accuracy: 0.9893


[0.03900706022977829, 0.9893048405647278]

In [38]:
func_saved_model_t62 = tf.keras.models.load_model('./kill_log_models/model_t62.hdf5')

In [39]:
func_saved_model_t62.evaluate(merged_testset)

24/24 [==============================] - 3s 110ms/step - loss: 0.0570 - accuracy: 0.9840


[0.05701206997036934, 0.9839572310447693]

### 6차 파인튜닝 결과

- 1차, 2차 평가 데이터의 정확도는 98.9%

기존 파인튜닝하기 전의 6차 모델의 1,2차 평가 데이터의 정확도가 97.3%였습니다. 파인튜닝을 통해 약 1.6%의 정확도 상승을 확인할 수 있습니다.  

파인튜닝을 통해서 기존 모델에 대한 고도화까지 진행해보았습니다. 이번에는 전이학습에 사용한 모델을 파라미터 수가 더 많은 VGG16로 변경하여 ResNet의 결과와 비교해보겠습니다.

## 7-1. 7차 학습


### VGG16

- 기존 모델 학습
- ResNet50보다 복잡한 VGG16 사용


### 1차 데이터셋

- kill 이미지 786
- non-kill 이미지 1832


### 1차 테스트셋

- kill 이미지 161
- non-kill 이미지 149


### 2차 데이터셋

- kill 이미지 1009
- non-kill 이미지 746


### 2차 테스트셋

- kill 이미지 252
- non-kill 이미지 186


### 7차 학습 특징


- 전이학습을 통해 기존 모델을 1,2차 데이터셋을 결합하여 전체 데이터에 대한 학습을 진행합니다.
- ROI을 추출한 뒤 Data Augmentation을 통해 데이터셋의 크기를 늘립니다.

In [40]:
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.layers import Dense, ReLU, BatchNormalization

In [41]:
base_model = VGG16(
    include_top = False,
    pooling = 'avg'
)
base_model.trainable = False
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

In [43]:
inputs = tf.keras.Input(shape=(190, 560, 3))
# 해당 모델은 학습에 사용하지 않겠다를 명시한다. 
x = base_model(inputs, training=False)
x =  Dense(32)(x)
x =  BatchNormalization()(x)
x =  ReLU()(x)
outputs = Dense(1, activation = 'sigmoid')(x)

func_model_t7 = tf.keras.Model(inputs, outputs)

# 학습 루프 정의 
func_model_t7.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

In [45]:
# model에 대한 체크포인트 생성 -> 가장 좋은 모델을 저장.
checkpoint = tf.keras.callbacks.ModelCheckpoint("./kill_log_models/model_t7.hdf5", monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='auto')

In [46]:
func_model_t7.fit(merged_cropped_augmented_train_set, validation_data = merged_valset, callbacks = [checkpoint], epochs = 20)

Epoch 1/20
129/129 [==============================] - 46s 301ms/step - loss: 0.3783 - accuracy: 0.8280 - val_loss: 0.0951 - val_accuracy: 0.9862

Epoch 00001: val_loss improved from inf to 0.09508, saving model to ./kill_log_models\model_t7.hdf5
Epoch 2/20
129/129 [==============================] - 20s 147ms/step - loss: 0.1337 - accuracy: 0.9718 - val_loss: 0.0565 - val_accuracy: 0.9931

Epoch 00002: val_loss improved from 0.09508 to 0.05647, saving model to ./kill_log_models\model_t7.hdf5
Epoch 3/20
129/129 [==============================] - 20s 148ms/step - loss: 0.1089 - accuracy: 0.9759 - val_loss: 0.0518 - val_accuracy: 0.9931

Epoch 00003: val_loss improved from 0.05647 to 0.05179, saving model to ./kill_log_models\model_t7.hdf5
Epoch 4/20
129/129 [==============================] - 20s 149ms/step - loss: 0.0924 - accuracy: 0.9778 - val_loss: 0.0564 - val_accuracy: 0.9931

Epoch 00004: val_loss did not improve from 0.05179
Epoch 5/20
129/129 [==============================] - 20s

In [47]:
func_model_t7.evaluate(merged_testset)

24/24 [==============================] - 8s 317ms/step - loss: 0.0936 - accuracy: 0.9693


[0.09361500293016434, 0.9692513346672058]

In [48]:
func_saved_model_t7 = tf.keras.models.load_model('./kill_log_models/model_t7.hdf5')

In [49]:
func_saved_model_t7.evaluate(merged_testset)

24/24 [==============================] - 3s 130ms/step - loss: 0.0628 - accuracy: 0.9853


[0.06276007741689682, 0.9852941036224365]

### 7차 학습의 결과

- 학습 데이터의 가장 높은 정확도는 99%

- 1,2차 평가 데이터의 정확도는 98.5%
 

전이학습을 위해 사용하였던 기학습 모델을 ResNet 모델에서 VGG 모델로 변경하였습니다. VGG 모델은 ResNet 모델보다 layer의 수는 적으나 파라미터 수가 더 많다는 차이가 존재합니다. 이전 학습에서도 파인튜닝으로 모델을 고도화하여 정확도 향상을 확인하였기 때문에 VGG 모델에 대해서도 파인튜닝을 진행합니다.

## 7-2. 7차 학습에 대한 파인튜닝

평가 데이터의 정확도가 충분히 높기 때문에 파인튜닝을 통해서 현재 모델을 고도화하는 방법을 선택할 수 있습니다.


### VGG16

- 7차 학습 모델 사용


### 1차 데이터셋 

- kill 이미지 786
- non-kill 이미지 1832


### 1차 테스트셋

- kill 이미지 161
- non-kill 이미지 149


### 2차 데이터셋

- kill 이미지 1009
- non-kill 이미지 746


### 2차 테스트셋

- kill 이미지 252
- non-kill 이미지 186


### 7차 파인튜닝 특징


- 전이학습을 위해 가중치 갱신이 불가능하도록 설정한 것을 파인튜닝을 위해 가능하도록 설정합니다.
- 1, 2차의 데이터셋을 결합합니다.
- ROI을 추출한 뒤 Data Augmentation을 통해 데이터셋의 크기를 늘립니다.

In [25]:
func_model_t72 = tf.keras.models.load_model('./kill_log_models/model_t7.hdf5')

In [26]:
func_model_t72.layers[1].trainable = True

In [27]:
func_model_t72.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 190, 560, 3)]     0         
_________________________________________________________________
vgg16 (Functional)           (None, 512)               14714688  
_________________________________________________________________
dense_2 (Dense)              (None, 32)                16416     
_________________________________________________________________
batch_normalization_1 (Batch (None, 32)                128       
_________________________________________________________________
re_lu_1 (ReLU)               (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 33        
Total params: 14,731,265
Trainable params: 14,731,201
Non-trainable params: 64
______________________________________________

In [28]:
# val_loss에 따른 빠른 종료 -> 빠른 구현을 위해서 선택적으로 적용할 수 있다.
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

# model에 대한 체크포인트 생성 -> 가장 좋은 모델을 저장. (모델을 저장하는 것은 ) 
checkpoint = tf.keras.callbacks.ModelCheckpoint("./kill_log_models/model_t72.hdf5", monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='auto')

In [29]:
func_model_t72.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),  # Low learning rate
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=['accuracy'],
)

In [31]:
func_model_t72.fit(merged_cropped_augmented_train_set, validation_data = merged_valset, callbacks = [checkpoint, early_stop], epochs = 20)

Epoch 1/20
129/129 [==============================] - 141s 735ms/step - loss: 0.0410 - accuracy: 0.9880 - val_loss: 0.0313 - val_accuracy: 0.9931

Epoch 00001: val_loss improved from inf to 0.03133, saving model to ./kill_log_models\model_t72.hdf5
Epoch 2/20
129/129 [==============================] - 65s 503ms/step - loss: 0.0303 - accuracy: 0.9907 - val_loss: 0.0298 - val_accuracy: 0.9954

Epoch 00002: val_loss improved from 0.03133 to 0.02981, saving model to ./kill_log_models\model_t72.hdf5
Epoch 3/20
129/129 [==============================] - 66s 504ms/step - loss: 0.0160 - accuracy: 0.9947 - val_loss: 0.0276 - val_accuracy: 0.9954

Epoch 00003: val_loss improved from 0.02981 to 0.02761, saving model to ./kill_log_models\model_t72.hdf5
Epoch 4/20
129/129 [==============================] - 66s 505ms/step - loss: 0.0139 - accuracy: 0.9956 - val_loss: 0.0304 - val_accuracy: 0.9954

Epoch 00004: val_loss did not improve from 0.02761
Epoch 5/20
129/129 [==============================] -

In [34]:
func_model_t72.evaluate(merged_testset)

24/24 [==============================] - 93s 4s/step - loss: 0.0407 - accuracy: 0.9920


[0.0407470278441906, 0.9919785857200623]

In [35]:
func_saved_model_t72 = tf.keras.models.load_model('./kill_log_models/model_t72.hdf5')

In [36]:
func_saved_model_t72.evaluate(merged_testset)

24/24 [==============================] - 38s 2s/step - loss: 0.0450 - accuracy: 0.9906


[0.045024774968624115, 0.990641713142395]

In [37]:
func_model_t72.save('./kill_log_models/best_model.hdf5')

### 7차 파인튜닝의 결과

- 학습 데이터의 가장 높은 정확도는 99%

- 1,2차 평가 데이터의 정확도는 100%
 
VGG 모델을 사용하여 구현한 모델의 정확도가 ResNet 모델을 사용하여 구현한 모델이 성능이 더 좋은 것을 확인할 수 있었습니다. 하지만 평가 데이터셋이 1,2차를 합치더라도 매우 적기 때문에 현재 모델의 성능을 판단하기 어렵습니다. 그러므로 새로 수집되는 리그오브레전드 이미지에 대해 모델을 적용해봄으로써 현재 모델의 성능을 판단해보겠습니다.